In [1]:
using Random
using Distributions
using DataFrames, CSV

# compile local files
include("statistics.jl")
include("simulation.jl")
include("data.jl")
include("util.jl")

using .TestStatistics
using .Simulation
using .Data

In [6]:
"""
CONFIG
"""

pooled = true
alpha = 0.05
isTwoSided = false

# data
B  = 100    # num. coverage probabilities per boxplot
S  = 1800   # num. samples per coverage probability
nx = 8      # size of group 1
ny = 8      # size of group 2
px, py = partition(nx, ny)
dtype = Float32

# distribution settings
Random.seed!(123)

"""
distrTypeX = Gamma
shapeX = dtype.(rand(Uniform(1, 5), B))
scaleX = 4 * shapeX
paramsX = (shapeX, scaleX)

distrTypeY = Gamma
shapeY = dtype.(rand(Uniform(1, 5), B))
scaleY = 4 * shapeY
paramsY = (shapeY, scaleY)
"""

distrTypeX = Exponential
lambdaX = dtype.(rand(Uniform(1, 20), B))
paramsX = (lambdaX,)

distrTypeY = Exponential
lambdaY = dtype.(rand(Uniform(1, 20), B))
paramsY = (lambdaY,)

(Float32[19.129026, 17.072006, 12.148226, 3.31444, 3.6175585, 1.9653575, 8.342362, 17.082706, 3.3472242, 15.060385  …  10.533955, 18.875303, 10.426724, 12.35601, 3.468512, 8.742228, 11.295829, 15.747598, 2.3619258, 18.72099],)

In [7]:
# generate data
x, y, wide, narrow, deltas, distrX, distrY = generateData(B, S, nx, ny, pooled,
                                                          distrTypeX, paramsX,
                                                          distrTypeY, paramsY)
@show size(x)
@show size(y)
@show size(wide)
@show size(narrow)
@show deltas[1]
@show distrX[1]
@show distrY[1]

size(x) = (100, 1800, 8)
size(y) = (100, 1800, 8)
size(wide) = (100, 1800)
size(narrow) = (100, 1800)
deltas[1] = -0.9093342f0
distrX[1] = Exponential{Float32}(θ=18.219692f0)
distrY[1] = Exponential{Float32}(θ=19.129026f0)


Exponential{Float32}(θ=19.129026f0)

In [8]:
# wrap constant arguments into an iterable object
# this allows each batch to have local access

struct Args
    px::AbstractMatrix{Int}
    py::AbstractMatrix{Int}
    pooled::Bool
    alpha::Float32
    alt_lo::Alternative
    alt_hi::Alternative
end

Base.Broadcast.broadcastable(obj::Args) = Ref(obj)  # make objects of this type iterable

In [9]:
for isTwoSided in [true, false]
    alpha_temp = alpha
    alt_lo = alt_hi = Simulation.twoSided
    
    if !isTwoSided
        alpha_temp = alpha / 2
        alt_lo = greater
        alt_hi = smaller
    end
        
    for pooled in [true, false]
        args = Args(px, py, pooled, alpha_temp, alt_lo, alt_hi)
        
        results = coverage.(eachslice(x, dims=1),
                            eachslice(y, dims=1),
                            eachrow(wide),
                            eachrow(narrow),
                            deltas,
                            args)
        
        # convert results to DataFrame
        probs  = [x for (x, _) in results]
        widths = [x for (_, x) in results]
        df = DataFrame(prob=probs, width=widths, distrX=distrX, distrY=distrY)

        # save DataFrame as .csv
        filename = (isTwoSided ? "twoSided" : "oneSided") * "_" *
                   (pooled ? "" : "un") * "pooled" * "_" * string(args.alpha) * ".csv"
        
        #CSV.write("../results/" * filename, df)
        CSV.write(filename, df)
    end
end